# Question 1
#### Write a Python function that takes the name of a star and returns a galpy Orbit object (setup with the star's position, distance, proper motion, and radial velocity). Apply to: GJ 440, Lacaille 8760, Vega. Check that you are finding the right star.

In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from galpy.orbit import Orbit
from numpy.ma import is_masked

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In order to search the Gaia catalogue by star name, I am first using astroquery to query the SIMBAD catalogue by star name. Using the star's position, proper motion, parallax, and radial velocity (if available) from SIMBAD, I then query the Gaia catalogue using the EPOCH_PROP_POS function for ADQL provided by the Gaia archive. This returns the star's RA and Dec corrected for the epoch. Using these corrected coordinates, the function queries Gaia for the star with a cone search in a small radius (by default 1 arcsec). If the star is found in Gaia, then the data is used to generate a galpy.orbit.Orbit object. If no data for the star is found, then data from SIMBAD is used instead. If the radial velocity of the star is not available in Gaia, then the function first checks if it is available in SIMBAD, and failing that defaults to 20.0 km/s (the approximate speed of the Sun with respect to the local standard of rest).

In [2]:
def star_to_orbit(star_name, radius = 1.0*u.arcsec):
    """ 
    Given the name of a star, returns a galpy.orbit.Orbit object for that 
    star using data from the Gaia DR2 catalogue. If no data on the star can
    be found from Gaia, then data from SIMBAD is used instead.
    """
    # Get data on the star from SIMBAD, for use in epoch adjustment
    custom_simbad = Simbad()
    custom_simbad.add_votable_fields('pmra', 'pmdec', 'plx', 'rv_value')
    simbad_table = custom_simbad.query_object(star_name)
    
    # Convert ra and dec from SIMBAD into a SkyCoord object
    # to transform into degrees
    simbad_coord = SkyCoord(ra = simbad_table['RA'][0], 
                            dec = simbad_table['DEC'][0],
                            unit = (u.hourangle, u.deg))
    
    # Store the SIMBAD values in a tuple of the form 
    # (ra, dec, plx, pmra, pmdec, rv)
    simbad_vals = (simbad_coord.ra.value,
                   simbad_coord.dec.value,
                   simbad_table['PLX_VALUE'][0],
                   simbad_table['PMRA'][0],
                   simbad_table['PMDEC'][0],
                   simbad_table['RV_VALUE'][0])
    
    # Circle radius to query Gaia, in degrees
    radius_deg = radius.to(u.deg)
    
    # Replace masked values from the SIMBAD table with 0.0 for epoch adjustment
    epoch_prop_vals = [val if not is_masked(val) else 0.0 for val in simbad_vals]
    
    # Perform a cone search with epoch adjustment
    query =  """
             SELECT source_id, ra, dec, pmra, pmdec, parallax, radial_velocity
             FROM gaiadr2.gaia_source
             WHERE 1=CONTAINS(
             POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),
             CIRCLE('ICRS', 
             COORD1(EPOCH_PROP_POS({0},{1},{2},{3},{4},{5},2000,2015.5)),
             COORD2(EPOCH_PROP_POS({0},{1},{2},{3},{4},{5},2000,2015.5)), {6}))
             """.format(*epoch_prop_vals, radius_deg.value)
             
    # Perfrom the cone search and get the result in a table
    job = Gaia.launch_job_async(query)
    gaia_table = job.get_results()
    
    # If no Gaia entries are found, use the SIMBAD entries
    if len(gaia_table) != 0:
        ra = gaia_table['ra'][0]
        dec = gaia_table['dec'][0]
        plx = gaia_table['parallax'][0]
        pmra = gaia_table['pmra'][0]
        pmdec = gaia_table['pmdec'][0]
        rv = gaia_table['radial_velocity'][0]
    else:
        ra, dec, plx, pmra, pmdec, rv = simbad_vals
        
    if is_masked(rv) and is_masked(simbad_vals[-1]):
        # Set rv to 20.0 if not available in either the Gaia or SIMBAD tables
        rv = 20.0
    elif is_masked(rv):
        # Use the SIMBAD rv if not available in GAIA
        rv = simbad_vals[5]
        
    # Generate and return a galpy.orbit.Orbit object
    orbit = Orbit(vxvv = (ra, dec, 1/plx, pmra, pmdec, rv), radec = True)
    return orbit

We can apply this function to the example stars GJ 440, Lacaille 8760, and Vega

In [3]:
stars = ["GJ 440", "Lacaille 8760", "Vega"]
orbits = [star_to_orbit(star) for star in stars]

Query finished.
Query finished.
Query finished.


We can check that the position and velocity values for each star are correct.

In [4]:
for i in range(len(stars)):
    orbit_vals = (stars[i], 
                  orbits[i].ra(), 
                  orbits[i].dec(),
                  1/orbits[i].dist(),
                  orbits[i].pmra(),
                  orbits[i].pmdec(),
                  orbits[i].vlos())
    orbit_str = ("star: {}\n"
    "ra (deg): {}\n"
    "dec (deg): {}\n"
    "plx (arcsec): {}\n"
    "pmra (mas/yr): {}\n"
    "pmdec (mas/yr): {}\n"
    "rv (km/s): {}\n").format(*orbit_vals)
    print(orbit_str)

star: GJ 440
ra (deg): 176.4557771399642
dec (deg): -64.84300499994973
plx (arcsec): 215.7372575196972
pmra (mas/yr): 2661.594499440631
pmdec (mas/yr): -344.84677507977284
rv (km/s): 20.000000000029228

star: Lacaille 8760
ra (deg): 319.29559940778796
dec (deg): -38.872297249579816
plx (arcsec): 251.82949243678786
pmra (mas/yr): -3258.5531275238395
pmdec (mas/yr): -1145.395729188883
rv (km/s): 20.561701480874195

star: Vega
ra (deg): 279.23473458332927
dec (deg): 38.78368888887407
plx (arcsec): 130.2299999999866
pmra (mas/yr): 200.9399999999357
pmdec (mas/yr): 286.2299999998336
rv (km/s): -20.600000000003046



Notice that the radial velocity for GJ 440 was not available in the Gaia dr2 catalogue nor in SIMBAD, and it therefore defaulted to 20.0 km/s. Moreover, Vega does not appear to be present in the Gaia dr2 release, and therefore the function used SIMBAD to get the data for Vega.